In [1]:
!python --version

Python 3.11.10


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

import json

# Initialize model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct" #3.8
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/jupyter/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.45s/it]


In [3]:
!pip install datasets
from datasets import load_dataset

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Define a function to get activations directly from the forward pass
def get_middle_layer_residual(model, inputs, layer_idx=16):
    residuals = []
    for i, layer in enumerate(model.model.layers):
        inputs = layer(inputs)
        if i == layer_idx:
            # Capture the output after the 16th layer
            residuals = inputs[0]  # Adjust indexing based on layer output format
            break
    return residuals

In [ ]:
# [batch, 100 * 4096]
# 1x 4096 -> 1x409610?
# 4096*4096*10*2
# 335,544,320

In [4]:
# Select the middle layer for residuals (e.g., for a 12-layer model, pick layer 6)
# Prepare storage
batch_size = 32
samples_per_file = 5000
max_samples = 100000
sample_count = 0
file_count = 1
data_batch = []

In [5]:
sentence = ["Hello how are you", "I am good thanks a lot"]
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt").to("cuda")

In [6]:
#WIP Need to test 
#Purpose - Given a tokenizer output and model output - Stores the word and corresponding vector to a dictionary
def get_word_vector_pairs(tokenizer_output, model_output):
    wordVectorPairs = []
    for i in range(len(inputs['input_ids'])):
        masked_input = (inputs['input_ids'][i] * inputs['attention_mask'][i]).cpu().tolist()
        print(masked_input)
        for j, word in enumerate(masked_input):
            print(word)
            print(SSD)
            if word == 0:
                continue
            else:
                wordVectorPairs.append(word, model_output[i][j])
    return wordVectorPairs
                
            

In [ ]:
# Process dataset in batches
for i in range(0, len(dataset), batch_size):
    if sample_count >= max_samples:
        break

    # Select a batch of samples
    batch = dataset[i : i + batch_size]
    texts = [sample["text"] for sample in batch]

    # Tokenize the batch of text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to("cuda")

    # Get the activations for the middle layer
    with torch.no_grad():
        residuals = get_middle_layer_residual(model, inputs)

    # Append text and residuals for each item in the batch
    for j, text in enumerate(texts):
        data_batch.extend(get_word_vector_pairs(inputs, residuals))
        #data_batch.append({"text": text, "residual": residuals[j].cpu().numpy().tolist()})
        sample_count += 1

    # Save batch to disk every 5,000 samples
    if sample_count % samples_per_file == 0 and data_batch:
        with open(f"residual_data_batch_{file_count}.jsonl", "w") as f:
            for entry in data_batch:
                f.write(json.dumps(entry) + "\n")
        data_batch.clear()
        file_count += 1

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from datasets import load_dataset
# import json

# Initialize model and tokenizer
# model_name = "microsoft/Phi-3-mini-4k-instruct"
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
## HOOK Approach 

# Select the middle layer for residuals (e.g., for a 12-layer model, pick layer 6)
layer_to_capture = 16
batch_size = 12  # Tweak based on GPU memory availability
samples_per_file = 5000
max_samples = 100000

# Create hooks for the residuals
residuals = []

def hook_fn(module, input, output):
    residuals.append(output.detach().cpu())

# Register hook
layer = model.model.layers[layer_to_capture]
hook_handle = layer.register_forward_hook(hook_fn)

# Load Pile dataset
dataset = load_dataset("the_pile", split="train")

# Prepare storage
sample_count = 0
file_count = 1
data_batch = []

# Process dataset in batches
for i in range(0, len(dataset), batch_size):
    if sample_count >= max_samples:
        break

    # Select a batch of samples
    batch = dataset[i : i + batch_size]
    texts = [sample['text'] for sample in batch]
    
    # Tokenize the batch of text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to("cuda")

    # Run forward pass to capture residuals for the entire batch
    with torch.no_grad():
        model(**inputs)

    # Append text and residuals for each item in the batch
    for j, text in enumerate(texts):
        if j < len(residuals):
            data_batch.append({"text": text, "residual": residuals[j].numpy().tolist()})
            sample_count += 1

    residuals.clear()  # Clear residuals to save memory

    # Save batch to disk every 5,000 samples
    if sample_count % samples_per_file == 0 and data_batch:
        with open(f"residual_data_batch_{file_count}.jsonl", "w") as f:
            for entry in data_batch:
                f.write(json.dumps(entry) + "\n")
        data_batch.clear()
        file_count += 1

# Remove the hook after processing
hook_handle.remove()

In [ ]:

# Define a function to get activations directly from the forward pass
def get_middle_layer_residual(model, inputs, layer_idx=16):
    residuals = []
    for i, layer in enumerate(model.model.layers):
        inputs = layer(inputs)
        if i == layer_idx:
            # Capture the output after the 16th layer
            residuals = inputs[0]  # Adjust indexing based on layer output format
            break
    return residuals

# Load Pile dataset
dataset = load_dataset("the_pile", split="train")

# Prepare storage
batch_size = 64
samples_per_file = 5000
max_samples = 100000
sample_count = 0
file_count = 1
data_batch = []

